<h2 style="color:#F3C969;">Project import statments</h2>

In [10]:
import geopandas as gpd
import random
import string

ImportError: DLL load failed while importing _network: The specified module could not be found.

In [8]:

# Get the active polygon layer
dmaBoundary = gpd.read_file("./GeoPandasSourceFIles/DMA Boundary V3.gpkg")

# Get the raster layer
rasterLayer = gpd.read_file("./GeoPandasSourceFiles/AAE.tif")


def alphaNumGen(size):
    # Get letters as a list
    letters = string.ascii_letters + string.digits
    # Return a random string of chars
    return ''.join(random.choices(letters, k=size))


# Create a point layer from the values of the raster layer
pointLayer = rasterLayer.to_points()
pointLayer.columns = ['x', 'y', 'elevation']

# Join the point layer to the original polygon layer
joinedLayer = dmaBoundary.merge(pointLayer, on='DMA')

# Create a new field called "id" and add it to joinedLayer layer
joinedLayer['id'] = joinedLayer['DMA'] + alphaNumGen(5)

# Classify features based on DMA
dmaDict = {}
for feature in joinedLayer.itertuples():
    dma = feature.DMA
    if dma not in dmaDict:
        dmaDict[dma] = []
    dmaDict[dma].append(feature)

# Find the features with the biggest and smallest elevation values
outputFeatures = []
for dma in dmaDict:
    minFeature = min(dmaDict[dma], key=lambda f: f.elevation)
    maxFeature = max(dmaDict[dma], key=lambda f: f.elevation)

    # Find the feature with the closest elevation to the mean
    meanElevation = sum([f.elevation for f in dmaDict[dma]]) / len(dmaDict[dma])
    avgFeature = min(dmaDict[dma], key=lambda f: abs(f.elevation - meanElevation))

    # Create a new feature with the min, max, and average elevation values
    outputFeatures.extend([minFeature, maxFeature, avgFeature])

# Export the layer to a GeoPackage
outputGeoPackage = 'min_max_layer.gpkg'
outputFeatures.to_file(outputGeoPackage, driver='GPKG')


NameError: name 'gpd' is not defined